In [2]:
#필요한 import
import pandas as pd
import numpy as np

In [3]:
# 리스트들 생성하기
list1 = ['홍', '김', '이', '박', '최']
list2 = ['길동', '철수', '성계', '지성', '용수']
list3 = [23, 45, 99, 41, 48]

names_dic = {'family_name':list1, 'first_name':list2, 'age':list3}

names = pd.DataFrame(names_dic)
names



,family_name,first_name,age
0,홍,길동,23
1,김,철수,45
2,이,성계,99
3,박,지성,41
4,최,용수,48


> DF 생성 시 리스트 길이 맞춰야 함.

In [4]:
list(zip(list1, list2, list3))

[('홍', '길동', 23),
 ('김', '철수', 45),
 ('이', '성계', 99),
 ('박', '지성', 41),
 ('최', '용수', 48)]

In [5]:
list(zip(list1,list2))

[('홍', '길동'), ('김', '철수'), ('이', '성계'), ('박', '지성'), ('최', '용수')]

In [13]:
#혹은 이 방법도 가능합니다.

# names_dic = {'family_name':list1, 'first_name':list2, 'age':list3}

pd.DataFrame(list(zip(list1, list2, list3)), 
            columns=['familyName', 'firstName', 'age'])

,familyName,firstName,age
0,홍,길동,23
1,김,철수,45
2,이,성계,99
3,박,지성,41
4,최,용수,48


In [14]:
list4 = np.arange(0, 4) # 0, 1, 2, 3
list4

array([0, 1, 2, 3])

In [15]:
pd.DataFrame(list(zip(list1,list2,list4)),columns=['family','first','age'])

,family,first,age
0,홍,길동,0
1,김,철수,1
2,이,성계,2
3,박,지성,3


# zip 생성 후 DF 생성 vs 딕셔너리 생성 후 DF 생성 차이점

> zip 생성은 key 값 개수가 맞지 않아도 오류 x
> 딕셔너리는 오류 발생

In [16]:
#일부러 애러 내기

names_dic_short = {'family_name':list1, 'first_name':list2, 'age':list4}
pd.DataFrame(names_dic_short)


ValueError: All arrays must be of the same length

In [17]:
names.index

# index 값 확인 메소드

RangeIndex(start=0, stop=5, step=1)

In [49]:
names_dic = {'family_name':list1, 'first_name':list2, 'age':list3}

names = pd.DataFrame(names_dic, 
                     index=['1번학생', '2번학생', '3번학생', '4번학생', '5번학생'])
names

# index 지정

,family_name,first_name,age
1번학생,홍,길동,23
2번학생,김,철수,45
3번학생,이,성계,99
4번학생,박,지성,41
5번학생,최,용수,48


> index를 내가 아는 걸로 지정하는 이유
-> loc으로 index 시 편함

In [50]:
names.loc['1번학생','age']

23

# loc vs iloc

loc은 이름으로 인덱스 하기 때문에 끝까지 인덱스
iloc은 -1

In [51]:
names = pd.DataFrame(names_dic)
names.to_csv('names.csv',index = False)
new_names = pd.read_csv('names.csv')

# csv 저장 및 읽기

In [52]:
#new_names.drop(columns=new_names.columns[0])
#new_names.iloc[:, 1:]
new_names

,family_name,first_name,age
0,홍,길동,23
1,김,철수,45
2,이,성계,99
3,박,지성,41
4,최,용수,48


In [53]:
# pd.read_csv('names.csv',index_col=0)
new_names.iloc[:,0:]

,family_name,first_name,age
0,홍,길동,23
1,김,철수,45
2,이,성계,99
3,박,지성,41
4,최,용수,48


In [54]:
new_names.drop(columns=new_names.columns[0])

,first_name,age
0,길동,23
1,철수,45
2,성계,99
3,지성,41
4,용수,48


코드를 보면 첫 컬럼에 희안한 값이 들어가 있죠? 

이는 index 정보를 담고 있는 column 입니다.

이 지저분한 열을 처리하는 것엔 크게 몇 가지 방법이 있습니다.

먼저 저장할 때 index를 포함하지 않는 것이죠. 

> names.to_csv('names.csv', index=False)

아니면, 저 특정 열을 index 로 지정해서 처리하는 방법입니다.

index_col 은 index로 사용할 컬럼의 위치가 몇 번째인지 지정합니다.

> pd.read_csv('names.csv', index_col=0)


# 3. 대용량 csv 파일 읽기

첫 시간에 pandas는 인메모리 (in-memory), 즉 메모리에 모든 데이터를 불러와 한 PC에서 사용할 수 있다고 배웠습니다. 만약 메모리가 충분히 크다면 데이터를 불러와 처리하는 것이 어렵지 않겠지만, GB가 넘어가는 파일을 불러오게 되면 장담할 수 없습니다.

이런 경우 할 수 있는 행동은

1. 더 큰 메모리 (구매 or 클라우드)
2. 파일 작게 자르기 (piece by piece)
3. 메모리를 적게 소비하는 구조 만들기 
4. 디스크 형태의 구조를 사용하기 (dask)

여기서는 3번에 대해 배워보겠습니다.

이 절에서는 diamonds 데이터를 이용합니다.


# 메모리 적게 소비하는 구조 만들기

In [60]:
diamonds = pd.read_csv('data/diamonds.csv', nrows=1000)
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float64
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float64
 5   table    1000 non-null   float64
 6   price    1000 non-null   int64  
 7   x        1000 non-null   float64
 8   y        1000 non-null   float64
 9   z        1000 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 78.2+ KB


In [62]:
np.finfo('float16')

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

> resolution : 소수점 몇번재 자리까지?

In [63]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [58]:
diamonds.describe().T

,count,mean,std,min,25%,50%,75%,max
carat,1000.0,0.68928,0.195291,0.20,0.70,0.71,0.79,1.27
depth,1000.0,61.72280,1.758879,53.00,60.90,61.80,62.60,69.50
table,1000.0,57.73470,2.467946,52.00,56.00,57.00,59.00,70.00
price,1000.0,2476.54000,839.575620,326.00,2777.00,2818.00,2856.00,2898.00
x,1000.0,5.60594,0.625173,3.79,5.64,5.77,5.92,7.12
y,1000.0,5.59918,0.611974,3.75,5.63,5.76,5.91,7.05
z,1000.0,3.45753,0.389819,2.27,3.45,3.55,3.64,4.33


소수점 둘째 자리까지 쓰고 있으니 float16을 써도 전혀 무리가 없겠습니다.

그러면, 모든 float64를 16으로 바꿔서 결과를 알아봅시다.

read_csv 에서 dtype 을 아래 코드와 같이 설정할 수 있습니다.

In [26]:
diamonds2 = pd.read_csv('data/diamonds.csv', nrows=1000,
           dtype={
               'carat': np.float16,
               'depth': np.float16,
               'table': np.float16,
               'x': np.float16,
               'y': np.float16,
               'z': np.float16,
               'price': np.int16
           })
diamonds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   float16
 1   cut      1000 non-null   object 
 2   color    1000 non-null   object 
 3   clarity  1000 non-null   object 
 4   depth    1000 non-null   float16
 5   table    1000 non-null   float16
 6   price    1000 non-null   int16  
 7   x        1000 non-null   float16
 8   y        1000 non-null   float16
 9   z        1000 non-null   float16
dtypes: float16(6), int16(1), object(3)
memory usage: 37.2+ KB


In [67]:
round(37.2/78.2,2)
#소수점 둘째까지~

0.48

# describe object 형 묘사

In [74]:
diamonds2.describe(include='O')

# cut / color / clarity object 있음.

,cut,color,clarity
count,1000,1000,1000
unique,5,7,8
top,Ideal,E,SI1
freq,333,240,306


In [77]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [72]:
diamonds2.cut.value_counts()

Ideal        333
Premium      290
Very Good    226
Good          89
Fair          62
Name: cut, dtype: int64

In [73]:
diamonds2.color.value_counts()

E    240
F    226
G    139
D    129
H    125
I     95
J     46
Name: color, dtype: int64

In [78]:
diamonds2.clarity.value_counts()


SI1     306
VS2     218
VS1     159
SI2     154
VVS2     62
VVS1     58
I1       29
IF       14
Name: clarity, dtype: int64

> 모두 범주형이라서 이럴 땐 object 보다 category 가 더 메모리 절약됨.

In [29]:
diamonds3 = pd.read_csv('data/diamonds.csv', nrows=1000,
           dtype={
               'carat': np.float16,
               'depth': np.float16,
               'table': np.float16,
               'x': np.float16,
               'y': np.float16,
               'z': np.float16,
               'price': np.int16,
               'color': 'category',
               'cut': 'category',
               'clarity': 'category'
           })
diamonds3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float16 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float16 
 5   table    1000 non-null   float16 
 6   price    1000 non-null   int16   
 7   x        1000 non-null   float16 
 8   y        1000 non-null   float16 
 9   z        1000 non-null   float16 
dtypes: category(3), float16(6), int16(1)
memory usage: 17.6 KB


In [30]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']

diamonds4 = pd.read_csv('data/diamonds.csv', nrows=1000,
           dtype={
               'carat': np.float16,
               'depth': np.float16,
               'price': np.int16,
               'table': np.float16,
               'color': 'category',
               'cut': 'category',
               'clarity': 'category'
           },
            usecols=cols)
diamonds4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    1000 non-null   float16 
 1   cut      1000 non-null   category
 2   color    1000 non-null   category
 3   clarity  1000 non-null   category
 4   depth    1000 non-null   float16 
 5   table    1000 non-null   float16 
 6   price    1000 non-null   int16   
dtypes: category(3), float16(3), int16(1)
memory usage: 11.8 KB


In [83]:
cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price']

diamonds_iter = pd.read_csv('data/diamonds.csv', nrows=1000,
           dtype={
               'carat': np.float16,
               'depth': np.float16,
               'price': np.int16,
               'color': 'category',
               'cut': 'category',
               'clarity': 'category'
           },
            usecols=cols,
            chunksize=200 #한번에 200row씩 불러옴.
        )

def process(df):
    return f'processed {df.shape} shape of df {df.iloc[0, :]}'
# df 형태로 0번째 row 부터 쭉

for chunk in diamonds_iter:
    print(process(chunk))

processed (200, 7) shape of df carat      0.22998
cut          Ideal
color            E
clarity        SI2
depth         61.5
table         55.0
price          326
Name: 0, dtype: object
processed (200, 7) shape of df carat       0.700195
cut        Very Good
color              E
clarity          SI1
depth       59.90625
table           63.0
price           2777
Name: 200, dtype: object
processed (200, 7) shape of df carat      0.300049
cut            Good
color             H
clarity         SI1
depth       63.8125
table          55.0
price           554
Name: 400, dtype: object
processed (200, 7) shape of df carat      0.700195
cut         Premium
color             F
clarity         VS2
depth      59.40625
table          61.0
price          2838
Name: 600, dtype: object
processed (200, 7) shape of df carat      1.219727
cut         Premium
color             E
clarity          I1
depth      60.90625
table          57.0
price          2862
Name: 800, dtype: object


# 엑셀 파일 읽기/쓰기

In [32]:
diamonds.to_excel('dia.xlsx', sheet_name='test')

In [33]:
dia = pd.read_excel('dia.xlsx')
dia

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.54,Ideal,D,VVS2,61.4,52.0,2897,5.30,5.34,3.26
996,996,0.72,Ideal,E,SI1,62.5,55.0,2897,5.69,5.74,3.57
997,997,0.72,Good,F,VS1,59.4,61.0,2897,5.82,5.89,3.48
998,998,0.74,Premium,D,VS2,61.8,58.0,2897,5.81,5.77,3.58
